In [10]:
import requests

# Replace these with your actual API keys
TMDB_API_KEY = 'b6d14935172e78430fe2ba8db683007f'
OMDB_API_KEY = ' http://www.omdbapi.com/?i=tt3896198&apikey=f9fc18cf'
TMDB_BASE_URL = 'https://api.themoviedb.org/3'
OMDB_BASE_URL = 'http://www.omdbapi.com/'

# Function to get genres from TMDb
def get_genres():
    try:
        response = requests.get(f'{TMDB_BASE_URL}/genre/movie/list', params={'api_key': TMDB_API_KEY})
        response.raise_for_status()  # Raise an error for bad status codes
        genres = response.json().get('genres', [])
        return {genre['name'].lower(): genre['id'] for genre in genres}
    except requests.exceptions.RequestException as e:
        print(f"Error fetching genres: {e}")
    return {}

# Function to get movies by genre from TMDb
def get_movies_by_genre(genre_id):
    try:
        response = requests.get(f'{TMDB_BASE_URL}/discover/movie', params={'api_key': TMDB_API_KEY, 'with_genres': genre_id})
        response.raise_for_status()  # Raise an error for bad status codes
        movies = response.json().get('results', [])
        movie_list = []
        for movie in movies[:5]:
            movie_details = {
                'title': movie['title'],
                'imdb_id': get_imdb_id(movie['id'])
            }
            movie_details['imdb_rating'] = get_movie_rating(movie_details['imdb_id']) if movie_details['imdb_id'] else 'N/A'
            movie_list.append(movie_details)
        return movie_list
    except requests.exceptions.RequestException as e:
        print(f"Error fetching movies: {e}")
    return []

# Function to get the IMDb ID for a given TMDb ID
def get_imdb_id(tmdb_id):
    try:
        response = requests.get(f'{TMDB_BASE_URL}/movie/{tmdb_id}', params={'api_key': TMDB_API_KEY})
        response.raise_for_status()  # Raise an error for bad status codes
        movie = response.json()
        return movie.get('imdb_id')
    except requests.exceptions.RequestException as e:
        print(f"Error fetching IMDb ID for TMDb ID {tmdb_id}: {e}")
    return None

# Function to get the IMDb rating for a given IMDb ID
def get_movie_rating(imdb_id):
    try:
        response = requests.get(f'{OMDB_BASE_URL}', params={'apikey': OMDB_API_KEY, 'i': imdb_id})
        response.raise_for_status()  # Raise an error for bad status codes
        movie = response.json()
        return movie.get('imdbRating')
    except requests.exceptions.RequestException as e:
        print(f"Error fetching rating for IMDb ID {imdb_id}: {e}")
    return None

# Function to handle movie recommendations
def recommend_movies():
    genres = get_genres()
    if not genres:
        print("Failed to retrieve genres from TMDb.")
        return

    print("Available genres: " + ", ".join(genres.keys()))

    while True:
        genre = input("Please enter a movie genre (or type 'quit' to exit): ").strip().lower()
        if genre == 'quit':
            print("Goodbye!")
            break

        genre_id = genres.get(genre)
        if genre_id:
            movies = get_movies_by_genre(genre_id)
            if movies:
                print(f"Here are some {genre.capitalize()} movies you might like:")
                for movie in movies:
                    print(f" - {movie['title']} (IMDb Rating: {movie['imdb_rating']})")
            else:
                print(f"Sorry, I couldn't find any movies in the {genre.capitalize()} genre.")
        else:
            print(f"Sorry, I don't recognize the genre '{genre}'. Please try another genre.")

if __name__ == "__main__":
    recommend_movies()


Available genres: action, adventure, animation, comedy, crime, documentary, drama, family, fantasy, history, horror, music, mystery, romance, science fiction, tv movie, thriller, war, western
Please enter a movie genre (or type 'quit' to exit): comedy
Error fetching rating for IMDb ID tt21692408: 401 Client Error: Unauthorized for url: http://www.omdbapi.com/?apikey=+http%3A%2F%2Fwww.omdbapi.com%2F%3Fi%3Dtt3896198%26apikey%3Df9fc18cf&i=tt21692408
Error fetching rating for IMDb ID tt21235248: 401 Client Error: Unauthorized for url: http://www.omdbapi.com/?apikey=+http%3A%2F%2Fwww.omdbapi.com%2F%3Fi%3Dtt3896198%26apikey%3Df9fc18cf&i=tt21235248
Error fetching rating for IMDb ID tt9466114: 401 Client Error: Unauthorized for url: http://www.omdbapi.com/?apikey=+http%3A%2F%2Fwww.omdbapi.com%2F%3Fi%3Dtt3896198%26apikey%3Df9fc18cf&i=tt9466114
Error fetching rating for IMDb ID tt5779228: 401 Client Error: Unauthorized for url: http://www.omdbapi.com/?apikey=+http%3A%2F%2Fwww.omdbapi.com%2F%3Fi%